# NVFLARE Hierarchical Statistics Visualization

#### dependencies

To run examples, you might need to install the dependencies
* ipywidgets
* numpy
* pandas
* matplotlib
* jupyter

These are captured in the requirements.txt

In [ ]:
%pip install -r ../requirements.txt

## Hierarchical Statistics Visualization of Tabular Data
In this example, we demonstrate how to visualize the hierarchical results from the statistics of tabular data. The visualization requires json, pandas, matplotlib and ipywidgets modules. 

In [71]:
import ipywidgets as widgets
import json
import matplotlib.pyplot as plt
import pandas as pd

First, copy the resulting json file to the demo directory. In this example, resulting file is called hierarchical_stats.json. Then load the json file

In [72]:
with open('hierarchical_stats.json', 'r') as f:
    data = json.load(f)

### Overall Statistics

In this section, we demonstrate how to visualize hierarchical stats from the tabular data using Jupyter Widgets.

In [ ]:

def create_widgets_hierarchy(data, parent_key='', sep='_'):
    """Convert hierarchical JSON into nested widgets for display."""
    if isinstance(data, dict):
        items = []
        for k, v in data.items():
            if k == "Global" or k == "Local":
                flattened_data = []
                for metric, values in v.items():
                    for key, value in values["default_set"].items():
                        flattened_data.append({
                            "Metric": metric,
                            "Type": key,
                            "Value": value
                        })
                df = pd.DataFrame(flattened_data)
                df_pivot = df.pivot(index="Metric", columns="Type", values="Value").T
                output = widgets.Output()
                with output:
                    display(df_pivot)
                items.append((k, output))
            elif k == "Name":
                continue
            else:
                new_key = f"{parent_key}{sep}{k}" if parent_key else k
                items.append((k, create_widgets_hierarchy(v, new_key, sep=sep)))
        return widgets.Accordion(children=[v for k, v in items], titles=[k for k, v in items])
    elif isinstance(data, list):
        items = []
        for i, item in enumerate(data):
            new_key = f"{parent_key}{sep}{i}" if parent_key else str(i)
            items.append((item.get("Name"), create_widgets_hierarchy(item, new_key, sep=sep)))
        return widgets.Accordion(children=[v for k, v in items], titles=[k for k, v in items])
    else:
        return widgets.Text(value=str(data), disabled=True)


widget_hierarchy = create_widgets_hierarchy(data)
display(widget_hierarchy)

### Histogram Visualization


The following code demonstrates how the hierarchical histograms can be displayed using Jupyter Widgets for features like `percentage`.

In [ ]:
skip_stats = ["count", "sum", "min", "max", "mean", "var", "stddev", "failure_count", "Name"]
def create_widgets_hierarchy(data, parent_key='', sep='_'):
    """Filter and convert hierarchical JSON into nested widgets of histograms for display."""
    if isinstance(data, dict):
        items = []
        for k, v in data.items():
            if k == "histogram":
                # Convert the data to a DataFrame
                percentage_data = v["default_set"]["Percentage"]
                df_percentage = pd.DataFrame(percentage_data, columns=["Range_Start", "Range_End", "Count"])
                # Plot the histogram for Percentage data
                plt.figure(figsize=(6, 6))
                bar_width = 0.5
                plt.bar(df_percentage.index, df_percentage["Count"], width=bar_width, tick_label=[f"{row[0]}-{row[1]}" for row in percentage_data])
                plt.title("Percentage")
                plt.xlabel("Range")
                plt.ylabel("Count")
                output = widgets.Output()
                with output:
                    plt.tight_layout()
                    plt.show()
                items.append((k, output))
            elif k in skip_stats:
                continue
            else:
                new_key = f"{parent_key}{sep}{k}" if parent_key else k
                items.append((k, create_widgets_hierarchy(v, new_key, sep=sep)))
        return widgets.Accordion(children=[v for k, v in items], titles=[k for k, v in items])
    elif isinstance(data, list):
        items = []
        for i, item in enumerate(data):
            new_key = f"{parent_key}{sep}{i}" if parent_key else str(i)
            items.append((item.get("Name"), create_widgets_hierarchy(item, new_key, sep=sep)))
        return widgets.Accordion(children=[v for k, v in items], titles=[k for k, v in items])
    else:
        return widgets.Text(value=str(data), disabled=True)


widget_tree = create_widgets_hierarchy(data)
display(widget_tree)